In [96]:
import pandas as pd
import numpy as np
import textblob
import nltk

import re
from nltk.corpus import stopwords
import seaborn as sns
import matplotlib.pyplot as plt
nltk.download('stopwords')

In [97]:
data =pd.read_csv('../input/emotional-data/Emotion analysis using NLP -code/dataset/train.txt',sep = ';',names = ['Input','Sentiment'],encoding='utf-8')

In [98]:
data

In [99]:
data.Sentiment.unique()

In [100]:
data['Sentiment'].replace('anger', 0,inplace =True)
data['Sentiment'].replace('fear', 1,inplace =True)
data['Sentiment'].replace('joy', 2,inplace =True)
data['Sentiment'].replace('love', 3,inplace =True)
data['Sentiment'].replace('sadness', 4,inplace =True)
data['Sentiment'].replace('surprise', 5,inplace =True)

In [101]:
data

In [102]:
data.Sentiment.value_counts()

In [103]:
sns.countplot((data['Sentiment']))

In [104]:
data.info()

In [105]:
x= data.Input
y= data.Sentiment

In [106]:
y.shape
y=np.array(y)

In [107]:
y

In [108]:
from nltk.stem.porter import PorterStemmer
ps =PorterStemmer()

In [109]:
len(data)

In [110]:
words =[]
for i in range(0,len(x)):
    word = re.sub('[^a-zA-Z]', ' ',x[i])
    word = word.lower()
    word = word.split()
    word = [ps.stem(text) for text in word if not text in stopwords.words('english') ]
    
    word = ' '.join(word)
    words.append(word)
    

In [111]:

from keras_preprocessing.text import one_hot
from keras_preprocessing.sequence import pad_sequences

In [112]:
one_hot

In [113]:
one_hot_representation = [one_hot(value, 10000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,) for value in words]

In [114]:
one_hot_representation[0]

In [115]:
padding = pad_sequences(one_hot_representation,padding ='pre', maxlen = 30)

In [116]:
padding.shape

In [117]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,Bidirectional

In [118]:
model = Sequential()
model.add(Embedding(10000,40, input_length = 20))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(6,activation ='relu'))
model.compile(loss='sparse_categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

model.summary()

In [119]:
from keras.utils.vis_utils import plot_model
plot_model(model)

In [120]:
input_data = np.array(padding)
output_data = y

In [121]:
input_data

In [122]:
from sklearn.model_selection import train_test_split

x_train, x_test,y_train, y_test = train_test_split(input_data,output_data,test_size = 0.33,random_state=42)

In [123]:
y_train.shape

In [124]:
model.fit(x_train,y_train,validation_data =(x_test,y_test),epochs =10,batch_size =20)

In [125]:
#model testing 
def model_test(string):
    words =[]
    word = re.sub('[^a-zA-Z]', ' ',string)
    word = word.lower()
    word = word.split()
    word = [ps.stem(text) for text in word if not text in stopwords.words('english') ]
    
    word = ' '.join(word)
    words.append(word)
    
    one_hot_representation = [one_hot(value, 10000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,) for value in words]
    padding = pad_sequences(one_hot_representation,padding ='pre', maxlen = 30)
    
    model = Sequential()
    model.add(Embedding(10000,40, input_length = 20))
    model.add(Bidirectional(LSTM(100)))
    model.add(Dense(6,activation ='relu'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
    
    
    prediction = np.argmax(model.predict(padding), axis=1)
    
    print(prediction)
    
    if prediction ==0:
        print('person is angry')
    elif prediction ==1:
        print('person is in fear')
    elif prediction ==2:
        print('person is in joy')
    elif prediction ==3:
        print('person is in love')
    elif prediction ==4:
        print('person is sad')
    else:
        print('person is in surprise')
        
text = str(input('enter input : '))
model_test(text)
    



In [126]:
model.save('emotion_detection.h5')